# 🔮 cellspell — MongoDB Spell (Colab)

Run MongoDB queries directly in notebook cells using mongosh syntax.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sreent/jupyter-query-magics/blob/main/examples/colab_mongodb.ipynb)

## Setup: Install and Start MongoDB

In [ ]:
# Install MongoDB server
!wget -q http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb > /dev/null 2>&1
!wget -qO - https://www.mongodb.org/static/pgp/server-4.4.asc | apt-key add - > /dev/null 2>&1
!echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.4 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-4.4.list > /dev/null
!apt-get update -qq > /dev/null
!apt-get install -y -qq mongodb-org > /dev/null
!mkdir -p /data/db
!mongod --fork --logpath /var/log/mongodb.log --dbpath /data/db

# Test MongoDB is running
!mongo --quiet --eval 'print("MongoDB ready!")'

In [ ]:
# Install cellspell
!pip install "cellspell[mongodb]" -q

In [ ]:
%load_ext cellspell.mongodb

## Connect to MongoDB

In [ ]:
%mongodb mongodb://localhost:27017/demo

In [ ]:
# Show connection info
%mongodb

## Insert Sample Data

In [ ]:
%%mongodb
db.users.insertMany([
    {"name": "Alice", "age": 30, "city": "Bangkok", "role": "engineer"},
    {"name": "Bob", "age": 25, "city": "London", "role": "designer"},
    {"name": "Charlie", "age": 35, "city": "Bangkok", "role": "engineer"},
    {"name": "Diana", "age": 28, "city": "Tokyo", "role": "manager"},
    {"name": "Eve", "age": 32, "city": "London", "role": "engineer"}
])

## Query Data

In [ ]:
# Find all users
%%mongodb
db.users.find()

In [ ]:
# Find engineers
%%mongodb
db.users.find({"role": "engineer"})

In [ ]:
# Find users over 30, sort by age descending
%%mongodb
db.users.find({"age": {"$gt": 30}}).sort({"age": -1})

In [ ]:
# Projection — only name and city
%%mongodb
db.users.find({}, {"name": 1, "city": 1, "_id": 0})

In [ ]:
# First 3 users sorted by age
%%mongodb
db.users.find().sort({"age": 1}).limit(3)

In [ ]:
# Find one user
%%mongodb
db.users.findOne({"name": "Alice"})

## Aggregation

In [ ]:
# Count users per city
%%mongodb
db.users.aggregate([
    {"$group": {"_id": "$city", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
])

In [ ]:
# Average age by role
%%mongodb
db.users.aggregate([
    {"$group": {"_id": "$role", "avg_age": {"$avg": "$age"}}},
    {"$sort": {"avg_age": -1}}
])

## Other Operations

In [ ]:
# Count matching documents
%%mongodb
db.users.countDocuments({"city": "Bangkok"})

In [ ]:
# Distinct cities
%%mongodb
db.users.distinct("city")

In [ ]:
# Update a document
%%mongodb
db.users.updateOne({"name": "Alice"}, {"$set": {"age": 31}})

In [ ]:
# Verify the update
%%mongodb
db.users.findOne({"name": "Alice"})

## Access Results in Python

The last query result is stored in `_mongodb`.

In [ ]:
%%mongodb
db.users.find({}, {"name": 1, "city": 1, "_id": 0})

In [ ]:
# Access the result as a list of dicts
for user in _mongodb:
    print(f"{user['name']} lives in {user['city']}")

## Clean Up

In [ ]:
%%mongodb
db.users.drop()

In [ ]:
# Disconnect
%mongodb --disconnect